# Setup

In [ ]:
from gensim.models import Word2Vec

In [ ]:
!pip install emoji
!pip install contractions #for expanding texts like i'm to i am

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
import re
import nltk
import emoji
import contractions
nltk.download('stopwords') #for stopword
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
#/content/final_dataset.xlsx
tweets_data = pd.read_excel("/content/sample_data/final _dataset.xlsx")
tweets_data

,text,likeCount,label
0,I had sharp cramps and couldn’t walk to the me...,2255,1.0
1,@SwiggyCares @Swiggy @swiggydineout see screen...,0,-1.0
2,@SwiggyCares @Swiggy Where can I enter the vo...,0,0.0
3,"Being a woman is so tiring, Why I am getting m...",20,-1.0
4,@Swiggy @SwiggyCares @fssaiindia @jagograhakja...,2,-1.0
...,...,...,...
149,"Sorry Bengaluru fans, but we really can’t deli...",209,1.0
150,State food depts and @fssaiindia are failing t...,9,0.0
151,"Swiggy, OfBusiness investor Norwest wants 8 qu...",16,0.0
152,Someone from Hyderabad just ordered 20 surf ex...,52,1.0


In [ ]:
tweets_data=tweets_data.rename(columns={'full_text':'text'})

# Emoji Processing

In [ ]:
def emoji_to_text(text):
  return emoji.demojize(text)


# Remove the punctuations and extra spaces

In [ ]:
import re

In [ ]:
def remove_punctuations(string):
  #to remove everything except alphanumerics, space
  punc_removed = re.sub(r"[^\w\s]","",string) #\w for alphanumerics
  return punc_removed

#Tokenization

In [ ]:
tweets_data["text"] = tweets_data["text"].apply(lambda tweet:tweet.split())

## Removing the tokens with link texts in them

In [ ]:
def removeLinkText(tokens):
  for token in tokens:
    if token.startswith("http"):
      tokens.remove(token)
  return tokens

## Changing slangs and contractions to normal english

In [ ]:
contractions.slang_dict

In [ ]:
contractions.fix("I'm") #contraction

'I am'

In [ ]:
contractions.fix("idk") #slang

'i do not know'

In [ ]:
short_forms = contractions.slang_dict #slang_dict has all slangs and contractions

In [ ]:
def remove_contractions(tokens):
  temp = tokens.copy()
  for index,token in enumerate(temp):
    if token in short_forms.keys():
      #replacing the token with expanded form
      temp.remove(token)
      expanded = contractions.fix(token) #getting the expanded form

      end_point = len(expanded) - 1
      temp[index:index] = expanded.split()

  return temp

# Stopword removal

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")

In [ ]:
def remove_stopwords(tokens,stopwords):
  non_stopwords = [token for token in tokens if token.lower() not in stopwords]
  return non_stopwords

In [ ]:
tweets_data["text"] = tweets_data["text"].apply(remove_stopwords,args=([stopwords]))
tweets_data["text"][0]

['Disappointed',
 'quality',
 'fabric',
 'new',
 'X',
 'Clothing',
 'Company',
 'line']

# Converting words to vectors

In [ ]:
temp_tokens = tweets_data["text"]
model2 = Word2Vec(temp_tokens, min_count=1, vector_size=10,
                                window=5, sg=1)
def replace_w2v(tokens):
  vectors = []

  for token in tokens:
    vector = model2.wv.get_vector(token)
    vectors.append(vector)

  return vectors

In [ ]:
tweets_data["Vectors"] = temp_tokens.apply(replace_w2v)

In [ ]:
tweets_data

,text,likeCount,label
0,I had sharp cramps and couldn’t walk to the me...,2255,1.0
1,@SwiggyCares @Swiggy @swiggydineout see screen...,0,-1.0
2,@SwiggyCares @Swiggy Where can I enter the vo...,0,0.0
3,"Being a woman is so tiring, Why I am getting m...",20,-1.0
4,@Swiggy @SwiggyCares @fssaiindia @jagograhakja...,2,-1.0
...,...,...,...
149,"Sorry Bengaluru fans, but we really can’t deli...",209,1.0
150,State food depts and @fssaiindia are failing t...,9,0.0
151,"Swiggy, OfBusiness investor Norwest wants 8 qu...",16,0.0
152,Someone from Hyderabad just ordered 20 surf ex...,52,1.0


In [ ]:
tweets_data.to_pickle("processed.pkl")

# Lemmatization (not useful)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def lemmatize(tokens,lemma):
    lemmatized = [lemma.lemmatize(token,pos='v') for token in tokens]
    return lemmatized

In [ ]:
lemma = WordNetLemmatizer()
# lemma.lemmatize("smart","a")

In [ ]:
def preprocess(twitter_text):
    twitter_text=twitter_text.apply(emoji_to_text)
    twitter_text=twitter_text.apply(remove_punctuations)
    # There were spaces before and after punctuations,
    # Hence after removing punctutions, there are extra spaces
    twitter_text = twitter_text.str.replace("  "," ")

    twitter_text=twitter_text.apply(lambda tweet:tweet.split())
    twitter_text=twitter_text.apply(remove_contractions)
    twitter_text=twitter_text.apply(remove_stopwords,args=([stopwords]))
    twitter_text=twitter_text.apply(lemmatize,args=([lemma]))
    return twitter_text

In [ ]:
tweets_data['text']=preprocess(tweets_data['text'])


In [ ]:
tweets_data['Vectors']

0      [[0.009645574, 0.07209736, 0.05067467, 0.08300...
1      [[0.14039432, -0.014069298, 0.042351384, 0.080...
2      [[0.14039432, -0.014069298, 0.042351384, 0.080...
3      [[0.04289896, -0.0541499, -0.026614422, -0.080...
4      [[0.14411378, 0.0024326902, 0.2429342, 0.12210...
                             ...                        
149    [[0.09617797, 0.088233165, -0.05845391, -0.091...
150    [[-0.012910279, -0.08576691, -0.07093309, 0.05...
151    [[0.14411378, 0.0024326902, 0.2429342, 0.12210...
152    [[-0.08331807, -0.040497683, -0.005788045, -0....
153    [[-0.075908236, 0.054515596, -0.037766542, 0.0...
Name: Vectors, Length: 154, dtype: object